In [1]:
import numpy as np
import pandas as pd
from numpy import dtype
from collections import defaultdict
from datetime import datetime as dt
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline
pd.set_option('display.max_columns', None)

In [2]:
iter_csv = pd.read_csv('../data/processed/train-processed.csv.gz', skipinitialspace = True, 
                       iterator=True, chunksize=50000)

data = pd.concat([chunk for chunk in iter_csv], ignore_index=True)

In [3]:
data.head()

,ncodpers,pais_residencia,sexo,age,lat,lon,fecha_alta,ind_nuevo,antiguedad,indrel,ind_empleado_A,ind_empleado_B,ind_empleado_F,ind_empleado_N,ind_empleado_S,ult_fec_cli_1t,indrel_1mes,tiprel_1mes_A,tiprel_1mes_I,tiprel_1mes_P,tiprel_1mes_R,indresi,indext,indfall,cod_prov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,15889,ES,1.0,56.0,40.0,-4.0,790214400,0.0,255.0,1.0,0.0,0.0,1.0,0.0,0.0,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,28.0,1.0,326124.90,1.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0.0,0.0,0
1,15890,ES,1.0,63.0,40.0,-4.0,790214400,0.0,256.0,1.0,1.0,0.0,0.0,0.0,0.0,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,28.0,1.0,71461.20,1.0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1.0,1.0,1
2,15891,ES,0.0,59.0,40.0,-4.0,1438041600,0.0,246.0,0.0,0.0,0.0,0.0,1.0,0.0,1438732800,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,28.0,0.0,0.00,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,15892,ES,0.0,62.0,40.0,-4.0,790214400,0.0,256.0,1.0,0.0,0.0,1.0,0.0,0.0,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,28.0,1.0,430477.41,1.0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0.0,0.0,1
4,15893,ES,1.0,63.0,40.0,-4.0,875836800,0.0,256.0,1.0,0.0,0.0,0.0,1.0,0.0,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,28.0,1.0,430477.41,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0


In [4]:
def age_group(age):
    return 0 if age < 35 else 1

def hashcode(user):
    group = age_group(user.age) 
    return (group, user.sexo)

In [5]:
clusterid_userids = defaultdict(list)

def split_by_clusters(row):        
    userhash = hashcode(row)
    clusterid_userids[userhash].append(row.ncodpers)
    
_ = data.apply(split_by_clusters, axis=1)

In [6]:
clusters_sizes = []

for key, val in clusterid_userids.items():
    clusters_sizes.append((key, len(val)))

In [7]:
clusters_sizes

[((0, 1.0), 191853),
 ((1, 0.0), 206846),
 ((0, 0.0), 227191),
 ((1, 1.0), 323719)]

In [8]:
cluster_id = 0

data.loc[:,'cluster'] = 0

for ids in clusterid_userids.values():
    data.cluster[data.ncodpers.isin(ids)] = cluster_id
    cluster_id += 1
    

data.head()

/Users/petr/Applications/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,ncodpers,pais_residencia,sexo,age,lat,lon,fecha_alta,ind_nuevo,antiguedad,indrel,ind_empleado_A,ind_empleado_B,ind_empleado_F,ind_empleado_N,ind_empleado_S,ult_fec_cli_1t,indrel_1mes,tiprel_1mes_A,tiprel_1mes_I,tiprel_1mes_P,tiprel_1mes_R,indresi,indext,indfall,cod_prov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,cluster
0,15889,ES,1.0,56.0,40.0,-4.0,790214400,0.0,255.0,1.0,0.0,0.0,1.0,0.0,0.0,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,28.0,1.0,326124.90,1.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0.0,0.0,0,3
1,15890,ES,1.0,63.0,40.0,-4.0,790214400,0.0,256.0,1.0,1.0,0.0,0.0,0.0,0.0,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,28.0,1.0,71461.20,1.0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1.0,1.0,1,3
2,15891,ES,0.0,59.0,40.0,-4.0,1438041600,0.0,246.0,0.0,0.0,0.0,0.0,1.0,0.0,1438732800,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,28.0,0.0,0.00,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,1
3,15892,ES,0.0,62.0,40.0,-4.0,790214400,0.0,256.0,1.0,0.0,0.0,1.0,0.0,0.0,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,28.0,1.0,430477.41,1.0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0.0,0.0,1,1
4,15893,ES,1.0,63.0,40.0,-4.0,875836800,0.0,256.0,1.0,0.0,0.0,0.0,1.0,0.0,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,28.0,1.0,430477.41,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0,3


In [9]:
clean_data_without_id = data.drop(['ncodpers', 'pais_residencia', 'cluster'], axis=1)

x = clean_data_without_id.values #returns a numpy array
x_scaled = MinMaxScaler().fit_transform(x)

normalized = pd.DataFrame(x_scaled)

In [10]:
normalized.columns=list(clean_data_without_id.columns.values)
normalized.tail()

,sexo,age,lat,lon,fecha_alta,ind_nuevo,antiguedad,indrel,ind_empleado_A,ind_empleado_B,ind_empleado_F,ind_empleado_N,ind_empleado_S,ult_fec_cli_1t,indrel_1mes,tiprel_1mes_A,tiprel_1mes_I,tiprel_1mes_P,tiprel_1mes_R,indresi,indext,indfall,cod_prov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
949604,1.0,0.308642,0.764151,0.355072,1.0,1.0,0.999744,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
949605,0.0,0.172840,0.764151,0.355072,1.0,1.0,0.999744,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.788462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
949606,1.0,0.117284,0.764151,0.355072,1.0,1.0,0.999744,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.538462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
949607,0.0,0.253086,0.764151,0.355072,1.0,1.0,0.999744,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.750000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
949608,1.0,0.234568,0.764151,0.355072,1.0,1.0,0.999744,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.403846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
normalized_data = pd.concat([data[['cluster', 'ncodpers']], normalized], axis=1)
normalized_data.to_csv('../data/processed/train-normalized.csv.gz', index=False, compression='gzip')

In [12]:
normalized_data.head()

,cluster,ncodpers,sexo,age,lat,lon,fecha_alta,ind_nuevo,antiguedad,indrel,ind_empleado_A,ind_empleado_B,ind_empleado_F,ind_empleado_N,ind_empleado_S,ult_fec_cli_1t,indrel_1mes,tiprel_1mes_A,tiprel_1mes_I,tiprel_1mes_P,tiprel_1mes_R,indresi,indext,indfall,cod_prov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,3,15889,1.0,0.333333,0.764151,0.355072,0.000000,0.0,0.999999,1.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.538462,1.0,0.011287,0.333333,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,3,15890,1.0,0.376543,0.764151,0.355072,0.000000,0.0,1.000000,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.538462,1.0,0.002473,0.333333,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
2,1,15891,0.0,0.351852,0.764151,0.355072,0.960543,0.0,0.999990,0.0,0.0,0.0,0.0,1.0,0.0,0.982361,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.538462,0.0,0.000000,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,15892,0.0,0.370370,0.764151,0.355072,0.000000,0.0,1.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.538462,1.0,0.014898,0.333333,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
4,3,15893,1.0,0.376543,0.764151,0.355072,0.126954,0.0,1.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.538462,1.0,0.014898,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
